# Pre-processing solver dataset and output clean dataset which is the input for clustering.

## Import packages

In [1]:
import pandas as pd
import regex as re
import pickle
import nltk

#from thoth.lab import common
from thoth.lab import solver
from nltk.tokenize import TreebankWordTokenizer
from string import punctuation  
from IPython.display import display, Markdown
from tqdm import tqdm

In [2]:
pd.set_option('max_colwidth', 1000)

## Set environment variables to retrieve data from Ceph and map to dataframe 
### DO NOT Run this everytime. 

In [72]:
import os
os.environ['THOTH_S3_ENDPOINT_URL'] = 'https://s3.upshift.redhat.com/'
os.environ['THOTH_CEPH_BUCKET'] = 'thoth'
os.environ['THOTH_CEPH_HOST'] = 'https://s3.upshift.redhat.com/'
os.environ['THOTH_CEPH_BUCKET_PREFIX'] = 'data/thoth'
os.environ['THOTH_DEPLOYMENT_NAME'] = 'thoth-psi-stage'

Connect to thoth storage

In [26]:
from thoth.storages import SolverResultsStore
store = SolverResultsStore(region="us-east-1")
store.connect()

### All the result in solver report are described below:

- **environment,** information about the environment on which the package has being solved;
- **environment_packages,** information about external packages installed on the environment;
- **errors,** if the installation of a package was not succesfull there will be information stored for each package error;
    - **details,**
        - **command**,
        - **message**,
        - **return_code**,
        - **stderr**,
        - **stdout**,
        - **timeout**,
    - **index_url,** from where the package was download;
    - **package_name;**
    - **package_version;**
    - **is_provided_package,** flag for storing package;
    - **is_provided_package_version,** flag for storing package;
    - **type,** error type;
- **tree**, all the packages installed in the dependency tree and information about them;
    - **dependencies**
    - **metadata** of the package as taken from importlib_metadata;
    - **index_url** from where the package was download;
    - **package_name;**
    - **package_version;**
    - **sha256;**
- **unparsed** if there are packages in the tree that could not be parsed;
- **unresolved,** if there are packages in the tree that could not be solved;

### Currently grafana metrics from the SolverResultsStore show over 700k results. Each solver has around 20k - 30k results with solver error. 

This block filters for documents with solver errors(Retrieve solver reports from Ceph and save only if it has "errors" in it)


In [27]:
solver_reports_extracted_data = []
solver_errors = []

for document_id in tqdm(store.get_document_listing()):
    solver_document = store.retrieve_document(document_id=document_id)
    solver_report_extracted_data = solver.extract_data_from_solver_metadata(solver_document["metadata"])
    for k, v in solver_document["result"].items():
        solver_report_extracted_data[k] = v
        if k == "errors" and v:
            errors = solver.extract_errors_from_solver_result(v)
            for error in errors:
                error['solver'] = solver_report_extracted_data['solver']
                solver_errors.append(error)

In [69]:
solver_total = pd.DataFrame(solver_errors).reset_index()
solver_total.to_csv('error_data.csv', index=False)
len(solver_total)

91000

## Load Error data from csv file created in the above step

In [3]:
solver_total_errors_df = pd.read_csv('error_data.csv')

In [4]:
solver_total_errors_df.head(5)

,index,command,index_url,message,package_name,package_version,return_code,solver,stderr,stdout,timeout,type
0,0,"/home/solver/venv/bin/python3 -m pip install --force-reinstall --no-cache-dir --no-deps json2html==1.1.0 --index-url ""https://pypi.org/simple"" --trusted-host pypi.org",https://pypi.org/simple,"Command exited with non-zero status code (1): ERROR: Command errored out with exit status 1:\n command: /home/solver/venv/bin/python3 -c 'import sys, setuptools, tokenize; sys.argv[0] = '""'""'/tmp/pip-install-1eq417ji/json2html/setup.py'""'""'; __file__='""'""'/tmp/pip-install-1eq417ji/json2html/setup.py'""'""';f=getattr(tokenize, '""'""'open'""'""', open)(__file__);code=f.read().replace('""'""'\r\n'""'""', '""'""'\n'""'""');f.close();exec(compile(code, __file__, '""'""'exec'""'""'))' egg_info --egg-base /tmp/pip-install-1eq417ji/json2html/pip-egg-info\n cwd: /tmp/pip-install-1eq417ji/json2html/\n Complete output (5 lines):\n Traceback (most recent call last):\n File ""<string>"", line 1, in <module>\n File ""/tmp/pip-install-1eq417ji/json2html/setup.py"", line 17, in <module>\n long_description=open('README.md').read(),\n FileNotFoundError: [Errno 2] No such file or directory: 'README.md'\n ----------------------------------------\nERROR: Command errored out with...",json2html,1.1.0,1.0,fedora-31-py37,"ERROR: Command errored out with exit status 1:\n command: /home/solver/venv/bin/python3 -c 'import sys, setuptools, tokenize; sys.argv[0] = '""'""'/tmp/pip-install-1eq417ji/json2html/setup.py'""'""'; __file__='""'""'/tmp/pip-install-1eq417ji/json2html/setup.py'""'""';f=getattr(tokenize, '""'""'open'""'""', open)(__file__);code=f.read().replace('""'""'\r\n'""'""', '""'""'\n'""'""');f.close();exec(compile(code, __file__, '""'""'exec'""'""'))' egg_info --egg-base /tmp/pip-install-1eq417ji/json2html/pip-egg-info\n cwd: /tmp/pip-install-1eq417ji/json2html/\n Complete output (5 lines):\n Traceback (most recent call last):\n File ""<string>"", line 1, in <module>\n File ""/tmp/pip-install-1eq417ji/json2html/setup.py"", line 17, in <module>\n long_description=open('README.md').read(),\n FileNotFoundError: [Errno 2] No such file or directory: 'README.md'\n ----------------------------------------\nERROR: Command errored out with exit status 1: python setup.py egg_info Check...",Collecting json2html==1.1.0\n Downloading https://files.pythonhosted.org/packages/70/3b/a0866f9a1ca7f3347546614770221a0061a8ea0ac5b7df053c6e55273ac8/json2html-1.1.0.tar.gz\n,60.0,command_error
1,1,"/home/solver/venv/bin/python3 -m pip install --force-reinstall --no-cache-dir --no-deps guillotina==1.3.0 --index-url ""https://pypi.org/simple"" --trusted-host pypi.org",https://pypi.org/simple,"Command exited with non-zero status code (1): ERROR: Command errored out with exit status 1:\n command: /home/solver/venv/bin/python3 -u -c 'import sys, setuptools, tokenize; sys.argv[0] = '""'""'/tmp/pip-install-5y8hsn2x/guillotina/setup.py'""'""'; __file__='""'""'/tmp/pip-install-5y8hsn2x/guillotina/setup.py'""'""';f=getattr(tokenize, '""'""'open'""'""', open)(__file__);code=f.read().replace('""'""'\r\n'""'""', '""'""'\n'""'""');f.close();exec(compile(code, __file__, '""'""'exec'""'""'))' bdist_wheel -d /tmp/pip-wheel-l0mjgi64 --python-tag cp37\n cwd: /tmp/pip-install-5y8hsn2x/guillotina/\n Complete output (308 lines):\n running bdist_wheel\n running build\n running build_py\n creating build\n creating build/lib.linux-x86_64-3.7\n creating build/lib.linux-x86_64-3.7/guillotina\n copying guillotina/__init__.py -> build/lib.linux-x86_64-3.7/guillotina\n copying guillotina/addons.py -> build/lib.linux-x86_64-3.7/guillotina\n copying guillotina/annotations.py -> build/lib.linux-x86_64-3....",guillotina,1.3.0,1.0,fedora-31-py37,"ERROR: Command errored out with exit status 1:\n command: /home/solver/venv/bin/python3 -u -c 'import sys, setuptools, tokenize; sys.argv[0] = '""'""'/tmp/pip-install-5y8hsn2x/guillotina/setup.py'""'""'; __file__='""'""'/tmp/pip-install-5y8hsn2x/guillo

## Split the error components from the log message

In [5]:
def split_log(log_messages):
    log_messages = log_messages.split('\n')
    return log_messages

In [6]:
error_df = pd.DataFrame()
error_df['index'] = solver_total_errors_df['index']
error_df['command'] = solver_total_errors_df['command']
error_df['package_name'] = solver_total_errors_df['package_name']
error_df['package_version'] = solver_total_errors_df['package_version']
error_df['solver'] = solver_total_errors_df['solver']

In [7]:
error_df['message'] = solver_total_errors_df['message']
error_df['split_message']= solver_total_errors_df.apply(lambda row: split_log(row.message),axis = 1)

### Example of error logs:

In [8]:
solver_total_errors_df['message'][999]

'Command exited with non-zero status code (1):     ERROR: Command errored out with exit status 1:\n     command: /home/solver/venv/bin/python3 -c \'import sys, setuptools, tokenize; sys.argv[0] = \'"\'"\'/tmp/pip-install-6xprrgcy/mysqlclient/setup.py\'"\'"\'; __file__=\'"\'"\'/tmp/pip-install-6xprrgcy/mysqlclient/setup.py\'"\'"\';f=getattr(tokenize, \'"\'"\'open\'"\'"\', open)(__file__);code=f.read().replace(\'"\'"\'\\r\\n\'"\'"\', \'"\'"\'\\n\'"\'"\');f.close();exec(compile(code, __file__, \'"\'"\'exec\'"\'"\'))\' egg_info --egg-base /tmp/pip-install-6xprrgcy/mysqlclient/pip-egg-info\n         cwd: /tmp/pip-install-6xprrgcy/mysqlclient/\n    Complete output (10 lines):\n    /bin/sh: mysql_config: command not found\n    Traceback (most recent call last):\n      File "<string>", line 1, in <module>\n      File "/tmp/pip-install-6xprrgcy/mysqlclient/setup.py", line 17, in <module>\n        metadata, options = get_config()\n      File "/tmp/pip-install-6xprrgcy/mysqlclient/setup_posix.py"

In [9]:
error_df['split_message'][999]

['Command exited with non-zero status code (1):     ERROR: Command errored out with exit status 1:',
 '     command: /home/solver/venv/bin/python3 -c \'import sys, setuptools, tokenize; sys.argv[0] = \'"\'"\'/tmp/pip-install-6xprrgcy/mysqlclient/setup.py\'"\'"\'; __file__=\'"\'"\'/tmp/pip-install-6xprrgcy/mysqlclient/setup.py\'"\'"\';f=getattr(tokenize, \'"\'"\'open\'"\'"\', open)(__file__);code=f.read().replace(\'"\'"\'\\r\\n\'"\'"\', \'"\'"\'\\n\'"\'"\');f.close();exec(compile(code, __file__, \'"\'"\'exec\'"\'"\'))\' egg_info --egg-base /tmp/pip-install-6xprrgcy/mysqlclient/pip-egg-info',
 '         cwd: /tmp/pip-install-6xprrgcy/mysqlclient/',
 '    Complete output (10 lines):',
 '    /bin/sh: mysql_config: command not found',
 '    Traceback (most recent call last):',
 '      File "<string>", line 1, in <module>',
 '      File "/tmp/pip-install-6xprrgcy/mysqlclient/setup.py", line 17, in <module>',
 '        metadata, options = get_config()',
 '      File "/tmp/pip-install-6xprrgcy

In [45]:
solver_total_errors_df['message'][19]

'Command exited with non-zero status code (1): ERROR: Could not find a version that satisfies the requirement lupa==0.2 (from versions: 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 0.10, 0.11, 0.12, 0.13, 0.13.1, 0.14, 0.15, 0.16, 0.17, 0.18, 0.19, 0.20, 0.21, 1.0b1, 1.0, 1.0.1, 1.1, 1.2, 1.3, 1.4, 1.5, 1.6, 1.7, 1.8, 1.9)\nERROR: No matching distribution found for lupa==0.2\n'

In [11]:
error_df['split_message'][19]

['Command exited with non-zero status code (1): ERROR: Could not find a version that satisfies the requirement lupa==0.2 (from versions: 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 0.10, 0.11, 0.12, 0.13, 0.13.1, 0.14, 0.15, 0.16, 0.17, 0.18, 0.19, 0.20, 0.21, 1.0b1, 1.0, 1.0.1, 1.1, 1.2, 1.3, 1.4, 1.5, 1.6, 1.7, 1.8, 1.9)',
 'ERROR: No matching distribution found for lupa==0.2',
 '']

In [42]:
solver_total_errors_df['message'][2269]

'Command exited with non-zero status code (1):   ERROR: Command errored out with exit status 1:\n   command: /home/solver/venv/bin/python3 -u -c \'import sys, setuptools, tokenize; sys.argv[0] = \'"\'"\'/tmp/pip-install-4r6sh7lx/cryptacular/setup.py\'"\'"\'; __file__=\'"\'"\'/tmp/pip-install-4r6sh7lx/cryptacular/setup.py\'"\'"\';f=getattr(tokenize, \'"\'"\'open\'"\'"\', open)(__file__);code=f.read().replace(\'"\'"\'\\r\\n\'"\'"\', \'"\'"\'\\n\'"\'"\');f.close();exec(compile(code, __file__, \'"\'"\'exec\'"\'"\'))\' bdist_wheel -d /tmp/pip-wheel-isct6cer --python-tag cp37\n       cwd: /tmp/pip-install-4r6sh7lx/cryptacular/\n  Complete output (36 lines):\n  running bdist_wheel\n  running build\n  running build_py\n  creating build\n  creating build/lib.linux-x86_64-3.7\n  creating build/lib.linux-x86_64-3.7/cryptacular\n  copying cryptacular/__init__.py -> build/lib.linux-x86_64-3.7/cryptacular\n  creating build/lib.linux-x86_64-3.7/cryptacular/core\n  copying cryptacular/core/__init__.py

In [43]:
error_df['split_message'][2269]

['Command exited with non-zero status code (1):   ERROR: Command errored out with exit status 1:',
 '   command: /home/solver/venv/bin/python3 -u -c \'import sys, setuptools, tokenize; sys.argv[0] = \'"\'"\'/tmp/pip-install-4r6sh7lx/cryptacular/setup.py\'"\'"\'; __file__=\'"\'"\'/tmp/pip-install-4r6sh7lx/cryptacular/setup.py\'"\'"\';f=getattr(tokenize, \'"\'"\'open\'"\'"\', open)(__file__);code=f.read().replace(\'"\'"\'\\r\\n\'"\'"\', \'"\'"\'\\n\'"\'"\');f.close();exec(compile(code, __file__, \'"\'"\'exec\'"\'"\'))\' bdist_wheel -d /tmp/pip-wheel-isct6cer --python-tag cp37',
 '       cwd: /tmp/pip-install-4r6sh7lx/cryptacular/',
 '  Complete output (36 lines):',
 '  running bdist_wheel',
 '  running build',
 '  running build_py',
 '  creating build',
 '  creating build/lib.linux-x86_64-3.7',
 '  creating build/lib.linux-x86_64-3.7/cryptacular',
 '  copying cryptacular/__init__.py -> build/lib.linux-x86_64-3.7/cryptacular',
 '  creating build/lib.linux-x86_64-3.7/cryptacular/core',
 ' 

In [14]:
def split_log(log_messages):
    ids_with_different_log_pattern = []
    Error_info, command_info, cwd, Complete_output, ERROR, specific_error, exception = {}, {}, {}, {}, {}, {}, {}
    for idx, msg in enumerate(log_messages):
        msg = msg.replace("Error:\n", "Error:")
        sentences = [x.strip() for x in msg.split('\n')]
        for id, sent in enumerate(sentences):
            if re.match(r"^WARNING.*", sent):
                pass
            elif re.match(r"^ERROR: .*", sent):
                if idx in ERROR.keys() and sent not in ERROR[idx]:
                    pass
                else:
                    ERROR[idx] = [sent]
            elif re.match(r"^Command.*", sent):
                Error_info[idx] = sent
            elif re.match(r"^command.*", sent):
                if idx in command_info.keys() and sent not in command_info[idx]:
                    command_info[idx].append(sent)
                else:
                    command_info[idx] = [sent]
            elif re.match(r"^cwd.*", sent):
                if idx in cwd.keys() and sent not in cwd[idx]:
                    cwd[idx].append(sent)
                else:
                    cwd[idx] = [sent]
            elif re.match(r"^Complete output.*", sent):
                number_of_lines = re.findall(r'\d+', sent) 
                if idx in Complete_output.keys() and sent not in Complete_output[idx]:
                    pass
                else:
                    Complete_output[idx] = sentences[id:id+int(number_of_lines[0])+1]
                    for txt in Complete_output[idx]:
                        if re.match(r"^Exception.*", txt):
                            exception[idx] = [txt]
            elif re.compile(".*Error.*").findall(sent):
                if idx in specific_error.keys() and sent not in specific_error[idx]:
                    if re.match(r".*unable to execute.*", specific_error[idx][0]):
                        pass
                    else:
                        specific_error[idx].extend([sent])
                else:
                    specific_error[idx] = [sent]
            elif re.match(r".*unable to execute.*", sent):
                specific_error[idx] = [sent]
    return Error_info, command_info, cwd, Complete_output, ERROR, specific_error, exception

print(len(solver_total_errors_df['message']))

91000


In [15]:
Error_info, command_info, cwd, Complete_output, ERROR, specific_error, exception = split_log(solver_total_errors_df['message'])
print(len(Error_info), len(command_info), len(cwd), len(Complete_output), len(ERROR), len(specific_error), len(exception))

89950 69199 69197 69197 88945 62378 702


In [16]:
error_df['Error_info']= error_df['index'].map(Error_info)
error_df['command_info']= error_df['index'].map(command_info)
error_df['cwd']= error_df['index'].map(cwd)
error_df['Complete_output']= error_df['index'].map(Complete_output)
error_df['ERROR']= error_df['index'].map(ERROR)
error_df['Exception']= error_df['index'].map(exception)
error_df['specific_error']= error_df['index'].map(specific_error)

In [17]:
error_df[250:255]

,index,command,package_name,package_version,solver,message,split_message,Error_info,command_info,cwd,Complete_output,ERROR,Exception,specific_error
250,250,"/home/solver/venv/bin/python3 -m pip install --force-reinstall --no-cache-dir --no-deps tiddlyweb==0.9.82 --index-url ""https://pypi.org/simple"" --trusted-host pypi.org",tiddlyweb,0.9.82,fedora-31-py37,"Command exited with non-zero status code (1): ERROR: Command errored out with exit status 1:\n command: /home/solver/venv/bin/python3 -c 'import sys, setuptools, tokenize; sys.argv[0] = '""'""'/tmp/pip-install-xcj3y037/tiddlyweb/setup.py'""'""'; __file__='""'""'/tmp/pip-install-xcj3y037/tiddlyweb/setup.py'""'""';f=getattr(tokenize, '""'""'open'""'""', open)(__file__);code=f.read().replace('""'""'\r\n'""'""', '""'""'\n'""'""');f.close();exec(compile(code, __file__, '""'""'exec'""'""'))' egg_info --egg-base /tmp/pip-install-xcj3y037/tiddlyweb/pip-egg-info\n cwd: /tmp/pip-install-xcj3y037/tiddlyweb/\n Complete output (5 lines):\n Traceback (most recent call last):\n File ""<string>"", line 1, in <module>\n File ""/tmp/pip-install-xcj3y037/tiddlyweb/setup.py"", line 14, in <module>\n long_description=file(os.path.join(os.path.dirname(__file__), 'README')).read(),\n NameError: name 'file' is not defined\n ----------------------------------------\nERROR: Command errored ...","[Command exited with non-zero status code (1): ERROR: Command errored out with exit status 1:, command: /home/solver/venv/bin/python3 -c 'import sys, setuptools, tokenize; sys.argv[0] = '""'""'/tmp/pip-install-xcj3y037/tiddlyweb/setup.py'""'""'; __file__='""'""'/tmp/pip-install-xcj3y037/tiddlyweb/setup.py'""'""';f=getattr(tokenize, '""'""'open'""'""', open)(__file__);code=f.read().replace('""'""'\r\n'""'""', '""'""'\n'""'""');f.close();exec(compile(code, __file__, '""'""'exec'""'""'))' egg_info --egg-base /tmp/pip-install-xcj3y037/tiddlyweb/pip-egg-info, cwd: /tmp/pip-install-xcj3y037/tiddlyweb/, Complete output (5 lines):, Traceback (most recent call last):, File ""<string>"", line 1, in <module>, File ""/tmp/pip-install-xcj3y037/tiddlyweb/setup.py"", line 14, in <module>, long_description=file(os.path.join(os.path.dirname(__file__), 'README')).read(),, NameError: name 'file' is not defined, ----------------------------------------, ERROR: Command errored...",Command exited with non-zero status code (1): ERROR: Command errored out with exit status 1:,"[command: /home/solver/venv/bin/python3 -c 'import sys, setuptools, tokenize; sys.argv[0] = '""'""'/tmp/pip-install-xcj3y037/tiddlyweb/setup.py'""'""'; __file__='""'""'/tmp/pip-install-xcj3y037/tiddlyweb/setup.py'""'""';f=getattr(tokenize, '""'""'open'""'""', open)(__file__);code=f.read().replace('""'""'\r\n'""'""', '""'""'\n'""'""');f.close();exec(compile(code, __file__, '""'""'exec'""'""'))' egg_info --egg-base /tmp/pip-install-xcj3y037/tiddlyweb/pip-egg-info]",[cwd: /tmp/pip-install-xcj3y037/tiddlyweb/],"[Complete output (5 lines):, Traceback (most recent call last):, File ""<string>"", line 1, in <module>, File ""/tmp/pip-install-xcj3y037/tiddlyweb/setup.py"", line 14, in <module>, long_description=file(os.path.join(os.path.dirname(__file__), 'README')).read(),, NameError: name 'file' is not defined]",[ERROR: Command errored out with exit status 1: python setup.py egg_info Check the logs for full command output.],NaN,[NameError: name 'file' is not defined]
251,251,"/home/solver/venv/bin/python3 -m pip install --force-reinstall --no-cache-dir --no-deps trash-cli==0.11.0.dev-r199 --index-url ""https://pypi.org/simple"" --trusted-host pypi.org",trash-cli,0.11.0.dev-r199,fedora-31-py37,"Command exited with non-zero status code (1): ERROR: Command errored out with exit status 1:\n command: /home/solver/venv/bin/python3 -c 'import sys, setuptools, tokenize; sys.argv[0] = '""'""'/tmp/pip-install-9qiqitga/trash-cli/setup.py'""'""'; __file__='""'""'/tmp/pip-install-9qiqitga/trash-cli/setup.py'""'""';f=getattr(tokenize, '""'""'open'""'""', open)(__file__);code=f.re

## Prepare the data for clustering

In [18]:
clustering_data = {}
for idx, error in enumerate(error_df['specific_error']):
    if type(error) != list:
        if pd.isnull(error) and  type(error_df.iloc[idx]['ERROR']) == list:
            clustering_data[idx] = [error_df.iloc[idx]['ERROR'][0]]
        if type(error_df.iloc[idx]['ERROR']) != list:
            if pd.isnull(error_df.iloc[idx]['ERROR']):
                clustering_data[idx] = [error_df.iloc[idx]['Error_info']]
                if type(error_df.iloc[idx]['Error_info']) != list:
                    if pd.isnull(error_df.iloc[idx]['Error_info']):
                        clustering_data[idx] = [error_df.iloc[idx]['message']]        
    else:
        for sent in error:
            regex = re.compile("(\w\w*Error)") 
            match_object = regex.findall(sent)
            #clustering_data[idx] = match_object
            if match_object:
                clustering_data[idx] = match_object
            else:
                clustering_data[idx] = [sent]
            if clustering_data[idx] == ['PyFrame_FastToLocalsWithError']:
                clustering_data[idx] = [error_df.iloc[idx]['ERROR'][0]]
    if clustering_data[idx] == ['Command exited with non-zero status code (-9):']:
        clustering_data[idx] = ['Error Info not Available']
    if re.match(".*make: .* |.*CMake .*|.*zError.*|.*SWIG_Python_TypeError.*|.*CatchlibLZMAError.*|.*PyExc_MemoryError.*|.*ViZDoomError.*|.*PyExc_KeyError.*|.*icsneoGetError.*|.*LapackSrcNotFoundError.*", clustering_data[idx][0]):
        clustering_data[idx] = error_df.iloc[idx]['ERROR']

In [19]:
len(clustering_data)

91000

In [20]:
clustering_data

{0: ['FileNotFoundError'],
 1: ["unable to execute 'gcc': No such file or directory"],
 2: ["unable to execute 'gcc': No such file or directory"],
 3: ['NameError'],
 4: ['ERROR: No matching distribution found for mxnet-mkl==1.7.0b20200309'],
 5: ['SyntaxError'],
 6: ["unable to execute 'gcc': No such file or directory"],
 7: ['ModuleNotFoundError'],
 8: ['FileNotFoundError'],
 9: ['FileNotFoundError'],
 10: ['TypeError'],
 11: ['FileNotFoundError'],
 12: ['ERROR: No matching distribution found for cbapi==0.9.0'],
 13: ['SyntaxError'],
 14: ['FileNotFoundError'],
 15: ['SyntaxError'],
 16: ['NameError'],
 17: ['ModuleNotFoundError'],
 18: ['ERROR: Command errored out with exit status 1: python setup.py egg_info Check the logs for full command output.'],
 19: ['ERROR: No matching distribution found for lupa==0.2'],
 20: ['SyntaxError'],
 21: ['SyntaxError'],
 22: ['SyntaxError'],
 23: ['SyntaxError'],
 24: ['ERROR: Failed building wheel for hdbscan'],
 25: ["unable to execute 'gcc': No 

## Cleaning clustering data

In [21]:
_line_number = r'(at line[:]*\s*\d+)'
_url = r'(http[s]|root|srm|file)*:(//|/)(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
_filepath = "(/[a-zA-Z\./]*[\s]?)"
path_regex = re.compile(r'(\b\w+://)\S+(?=\s)')
file_regex = re.compile(r'(\b[f|F]ile( exists)?:?\s?)/\S+(?=\s)')
py_regex = re.compile(r'/?\b[-./_a-zA-Z0-9]+\.py\b')
long_regex = re.compile(r'[-/_a-zA-Z0-9]{25,}')

In [22]:
def remove_whitespaces(sentence):
    return " ".join(sentence.split())

def substitute_path(string):
    string = path_regex.sub(r'\1', string)
    string = py_regex.sub(r' ', string)
    string = file_regex.sub(r'\1', string)
    string = long_regex.sub(r'', string)
    return string

def cleaner(log_messages):
    clean_log = {}
    for key in log_messages:
        for item in log_messages[key]:
            item = re.sub(_line_number, "at line *", item)
            item = re.sub(_url, " ", item)
            item = re.sub(_filepath, " ", item)
            #item = re.sub("\d+", " ", item)
            #item = re.sub(r'\b\w\b', ' ', item)
            item = substitute_path(item)
            if key in clean_log.keys() and item not in clean_log[key]:
                clean_log[key] = clean_log[key] + ' ' + remove_whitespaces(item)
            else:
                clean_log[key] = remove_whitespaces(item)
    return clean_log

In [23]:
clustering_data = cleaner(clustering_data)

In [24]:
len(clustering_data)

91000

## Tokenization 

In [25]:
stemmer = nltk.PorterStemmer()
stop = punctuation + "``" + "''" + '""' + "/"
table = str.maketrans(stop, ' '*len(stop))

In [26]:
def tokenization(log_messages):
    tokenized = []
    for key, item in log_messages.items():
        item = item.replace(' Error','Error').strip()
        item = item.replace('Errno','').strip()
        item = item.replace('Error :','Error:').strip()
        item = item.replace('Exception:','').strip()
        if error_df['package_name'][key] in item.split():
            item = item.replace(error_df['package_name'][key],'').strip()
        if "interpreter:" in item.split():
            item = item.split("interpreter",1)[1]
        if "for"  in item.split():
            item = item.split("for",1)[0] 
        if "in" in item.split():
            if "in an" in item:
                item = item.split("in an",1)[1]
            else:
                item = item.split("in",1)[0]
        if "on" in item.split():
            item = re.split(" on ",item)[0] 
        if re.match(r"^ERROR:.* |^Error:.* |^error:.*", item):
            item = re.split("^ERROR: |^error: |^Error: |^error:Error:",item)[1].split('.')[0]
        if item.strip() == '':
            item = error_df.iloc[idx]['ERROR'][0].translate(table).strip()
        if "Command" in item:
            if "ERROR:" in item:
                item = item.split("ERROR:",1)[1] 
            if re.match(r".*Command errored out with exit status.*", item):
                item = "Check the logs"
        if "not found" in item:
            words = [stemmer.stem(word) for word in item.split()] 
            item = ''
            for word in words:
                item += word[0].upper() + word[1:]
            item += "Error"
        if ":" in item and item[0] != ":":
            item = item.split(":",1)[0] 
        if "JAVA_HOME" in item:
            item = 'JAVA HOME not set to a path containing the JDK'
        if re.match("(\w\w* error)", item):
            item = item.split()[0] + "Error"
        if "XML" in item:
            item = "cannot get pre-processor and compiler flags"
        item = item.translate(table).strip()
        tokenized.append(TreebankWordTokenizer().tokenize(item))
    cleaned_tokens = []
    for id, row in enumerate(tokenized):
        #cleaned_tokens.append([i for i in row if i.lower() not in stop and not i.lower().isnumeric()])
        cleaned_tokens.append(list(filter(None, [i
                                                 for i in row 
                                                 if i != error_df['package_name'][id]
                                                 and not i.lower().isnumeric()])))
    return cleaned_tokens

In [27]:
tokenized_clustering_data = tokenization(clustering_data)

In [28]:
tokenized_clustering_data

[['FileNotFoundError'],
 ['unable', 'to', 'execute', 'gcc'],
 ['unable', 'to', 'execute', 'gcc'],
 ['NameError'],
 ['No', 'matching', 'distribution', 'found'],
 ['SyntaxError'],
 ['unable', 'to', 'execute', 'gcc'],
 ['ModuleNotFoundError'],
 ['FileNotFoundError'],
 ['FileNotFoundError'],
 ['TypeError'],
 ['FileNotFoundError'],
 ['No', 'matching', 'distribution', 'found'],
 ['SyntaxError'],
 ['FileNotFoundError'],
 ['SyntaxError'],
 ['NameError'],
 ['ModuleNotFoundError'],
 ['Check', 'the', 'logs'],
 ['No', 'matching', 'distribution', 'found'],
 ['SyntaxError'],
 ['SyntaxError'],
 ['SyntaxError'],
 ['SyntaxError'],
 ['Failed', 'building', 'wheel'],
 ['unable', 'to', 'execute', 'gcc'],
 ['unable', 'to', 'execute', 'gcc'],
 ['FileNotFoundError'],
 ['SyntaxError'],
 ['SyntaxError'],
 ['SyntaxError'],
 ['ImportError'],
 ['SyntaxError'],
 ['HTTPError'],
 ['ModuleNotFoundError'],
 ['ModuleNotFoundError'],
 ['No', 'matching', 'distribution', 'found'],
 ['SyntaxError'],
 ['FileNotFoundError'],


## Save the cleaned data for clustering

In [29]:
error_df['clustering_data'] = error_df['index'].map(clustering_data)

In [30]:
error_df['tokenized_clustering_data'] = tokenized_clustering_data

In [31]:
error_df.head(20)

,index,command,package_name,package_version,solver,message,split_message,Error_info,command_info,cwd,Complete_output,ERROR,Exception,specific_error,clustering_data,tokenized_clustering_data
0,0,"/home/solver/venv/bin/python3 -m pip install --force-reinstall --no-cache-dir --no-deps json2html==1.1.0 --index-url ""https://pypi.org/simple"" --trusted-host pypi.org",json2html,1.1.0,fedora-31-py37,"Command exited with non-zero status code (1): ERROR: Command errored out with exit status 1:\n command: /home/solver/venv/bin/python3 -c 'import sys, setuptools, tokenize; sys.argv[0] = '""'""'/tmp/pip-install-1eq417ji/json2html/setup.py'""'""'; __file__='""'""'/tmp/pip-install-1eq417ji/json2html/setup.py'""'""';f=getattr(tokenize, '""'""'open'""'""', open)(__file__);code=f.read().replace('""'""'\r\n'""'""', '""'""'\n'""'""');f.close();exec(compile(code, __file__, '""'""'exec'""'""'))' egg_info --egg-base /tmp/pip-install-1eq417ji/json2html/pip-egg-info\n cwd: /tmp/pip-install-1eq417ji/json2html/\n Complete output (5 lines):\n Traceback (most recent call last):\n File ""<string>"", line 1, in <module>\n File ""/tmp/pip-install-1eq417ji/json2html/setup.py"", line 17, in <module>\n long_description=open('README.md').read(),\n FileNotFoundError: [Errno 2] No such file or directory: 'README.md'\n ----------------------------------------\nERROR: Command errored out with...","[Command exited with non-zero status code (1): ERROR: Command errored out with exit status 1:, command: /home/solver/venv/bin/python3 -c 'import sys, setuptools, tokenize; sys.argv[0] = '""'""'/tmp/pip-install-1eq417ji/json2html/setup.py'""'""'; __file__='""'""'/tmp/pip-install-1eq417ji/json2html/setup.py'""'""';f=getattr(tokenize, '""'""'open'""'""', open)(__file__);code=f.read().replace('""'""'\r\n'""'""', '""'""'\n'""'""');f.close();exec(compile(code, __file__, '""'""'exec'""'""'))' egg_info --egg-base /tmp/pip-install-1eq417ji/json2html/pip-egg-info, cwd: /tmp/pip-install-1eq417ji/json2html/, Complete output (5 lines):, Traceback (most recent call last):, File ""<string>"", line 1, in <module>, File ""/tmp/pip-install-1eq417ji/json2html/setup.py"", line 17, in <module>, long_description=open('README.md').read(),, FileNotFoundError: [Errno 2] No such file or directory: 'README.md', ----------------------------------------, ERROR: Command errored out wit...",Command exited with non-zero status code (1): ERROR: Command errored out with exit status 1:,"[command: /home/solver/venv/bin/python3 -c 'import sys, setuptools, tokenize; sys.argv[0] = '""'""'/tmp/pip-install-1eq417ji/json2html/setup.py'""'""'; __file__='""'""'/tmp/pip-install-1eq417ji/json2html/setup.py'""'""';f=getattr(tokenize, '""'""'open'""'""', open)(__file__);code=f.read().replace('""'""'\r\n'""'""', '""'""'\n'""'""');f.close();exec(compile(code, __file__, '""'""'exec'""'""'))' egg_info --egg-base /tmp/pip-install-1eq417ji/json2html/pip-egg-info]",[cwd: /tmp/pip-install-1eq417ji/json2html/],"[Complete output (5 lines):, Traceback (most recent call last):, File ""<string>"", line 1, in <module>, File ""/tmp/pip-install-1eq417ji/json2html/setup.py"", line 17, in <module>, long_description=open('README.md').read(),, FileNotFoundError: [Errno 2] No such file or directory: 'README.md']",[ERROR: Command errored out with exit status 1: python setup.py egg_info Check the logs for full command output.],NaN,[FileNotFoundError: [Errno 2] No such file or directory: 'README.md'],FileNotFoundError,[FileNotFoundError]
1,1,"/home/solver/venv/bin/python3 -m pip install --force-reinstall --no-cache-dir --no-deps guillotina==1.3.0 --index-url ""https://pypi.org/simple"" --trusted-host pypi.org",guillotina,1.3.0,fedora-31-py37,"Command exited with non-zero status code (1): ERROR: Command errored out with exit status 1:\n command: /home/solver/venv/bin/python3 -u -c 'import sys, setuptools, tokenize; sys.argv[0] = '""'""'/tmp/pip-install-5y8hsn2x/guillotina/setup.py'""'""'; __file__='""'""'/tmp/pip-install-5y8hsn2x/guillotina/se

In [32]:
#with open("solver-errors-clean-clustering-data.txt", "wb") as fp: 
 #   pickle.dump(error_df['tokenized_clustering_data'], fp)

In [33]:
error_df.to_csv('error-clean-data.csv', index=False)